## Construcción del modelo

Apalancados en la ingeniería de features, creamos un conjunto de clases a utilizar en la definición de pipelines, que nos permitan reproducir y modificar con facilidad los pasos de preprocesamiento, previos al entrenamiento de un modelo: 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sys
sys.path.append('src')
from feature_engineering.name_splitter import NameSplitter
from feature_engineering.owner_mapper import OwnerMapper
from feature_engineering.seat_rounder import SeatRounder
from feature_engineering.max_power_converter import MaxPowerConverter
from feature_engineering.engine_converter import EngineConverter
from feature_engineering.multiple_interative_imputer import MultipleIterativeImputer
from feature_engineering.multiple_one_hot_encoder import MultipleOneHotEncoder
from feature_engineering.torque_standardizer import TorqueStandardizer
from feature_engineering.mileage_converter import MileageConverter
from feature_engineering.model_dropper import ModelDropper

In [2]:
categorical_cols = ['fuel', 'seller_type', 'transmission', 'make']

# Definir pipelines para preprocesamiento
preprocess_pipeline = Pipeline(steps=[
    ('name_spliter', NameSplitter()),
    ('mileage_converter', MileageConverter()),
    ('engine_converter', EngineConverter()),
    ('max_power_converter', MaxPowerConverter()),
    ('torque_standardizer', TorqueStandardizer()),
    ('map_owner', OwnerMapper()),
    ('model_dropper', ModelDropper()),
])

preprocess_pipeline_with_one_hot_encoder = Pipeline(steps=[
    ('preprocess_pipeline', preprocess_pipeline),
    ('multiple_one_hot_encoder', MultipleOneHotEncoder(categorical_cols=categorical_cols)),
])

columns_to_drop_for_imputation = []

full_pipeline = Pipeline(steps=[
    ('multiple_iterative_imputer', MultipleIterativeImputer(columns_to_drop=columns_to_drop_for_imputation)),
    ('round_seats', SeatRounder()),
])

final_pipeline = Pipeline(steps=[
    ('preprocess_pipeline', preprocess_pipeline_with_one_hot_encoder),
    ('full_pipeline', full_pipeline),
])

final_pipeline_with_scaler = Pipeline(steps=[
    ('final_pipeline', final_pipeline),
    ('scaler', StandardScaler())
])

In [3]:
# Cargar y dividir los datos
data = pd.read_csv('../datasets/Car details v3.csv')

data["selling_price_log"] = np.log(data["selling_price"])

X = data.drop(columns=['selling_price', 'selling_price_log'])
y = data['selling_price']
y_log = data['selling_price_log']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=42)

In [4]:
# Ajustar y transformar los datos
X_train_processed = final_pipeline_with_scaler.fit_transform(X_train)
X_test_processed = final_pipeline_with_scaler.transform(X_test)

# Definir pipelines para preprocesamiento
preprocess_pipeline_log = Pipeline(steps=[
    ('name_spliter', NameSplitter()),
    ('mileage_converter', MileageConverter()),
    ('engine_converter', EngineConverter()),
    ('max_power_converter', MaxPowerConverter()),
    ('torque_standardizer', TorqueStandardizer()),
    ('map_owner', OwnerMapper()),
    ('model_dropper', ModelDropper()),
])

preprocess_pipeline_with_one_hot_encoder_log = Pipeline(steps=[
    ('preprocess_pipeline', preprocess_pipeline_log),
    ('multiple_one_hot_encoder', MultipleOneHotEncoder(categorical_cols=categorical_cols)),
])

full_pipeline_log = Pipeline(steps=[
    ('multiple_iterative_imputer', MultipleIterativeImputer(columns_to_drop=columns_to_drop_for_imputation)),
    ('round_seats', SeatRounder()),
])

final_pipeline_log = Pipeline(steps=[
    ('preprocess_pipeline', preprocess_pipeline_with_one_hot_encoder_log),
    ('full_pipeline', full_pipeline_log),
])

final_pipeline_with_scaler_log = Pipeline(steps=[
    ('final_pipeline', final_pipeline_log),
    ('scaler', StandardScaler())
])

X_train_processed_log = final_pipeline_with_scaler_log.fit_transform(X_train_log)
X_test_processed_log = final_pipeline_with_scaler_log.transform(X_test_log)

/Users/cbarril/posgrado/materias/aprendizaje_maquinas_I/tp_amq1_17co2024/amq1/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cbarril/posgrado/materias/aprendizaje_maquinas_I/tp_amq1_17co2024/amq1/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Veamos de usar un Ridge como primer modelo simple. Usaremos búsqueda de grilla para el hiperparámetro alpha:

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error


from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [6]:
from sklearn.linear_model import LinearRegression

regresion_lineal = LinearRegression()
regresion_lineal.fit(X_train_processed, y_train)

regresion_lineal_log = LinearRegression()
regresion_lineal_log.fit(X_train_processed_log, y_train)

LinearRegression()

In [7]:
# Predicciones para ambos casos
y_pred_original = regresion_lineal.predict(X_test_processed)

# Cálculo de métricas de error para los datos originales
mae_original = mean_absolute_error(y_test, y_pred_original)
rmse_original = np.sqrt(mean_squared_error(y_test, y_pred_original))
mape_original = mean_absolute_percentage_error(y_test, y_pred_original)
r2_original = r2_score(y_test, y_pred_original)

# Imprimir resultados
print("Resultados sin logaritmo:")
print(f"MAE de testeo fue: {mae_original}")
print(f"RMSE de testeo fue: {rmse_original}")
print(f"MAPE de testeo fue: {mape_original}")
print(f"R2 de testeo fue: {r2_original}")

Resultados sin logaritmo:
MAE de testeo fue: 163174.72357215974
RMSE de testeo fue: 336767.0735208581
MAPE de testeo fue: 0.46334656775548905
R2 de testeo fue: 0.8353629178595343


In [8]:
# Para los datos con logaritmo, debes invertir el logaritmo de las predicciones y del conjunto de test
y_pred_log = regresion_lineal_log.predict(X_test_processed_log)

# Invertir logaritmo tanto para predicciones como para y_test para hacer comparaciones justas
y_pred_log_inv = np.exp(y_pred_log)
y_test_inv = np.exp(y_test_log)

# Cálculo de métricas de error para los datos con log (ahora comparando con los valores originales)
mae_log = mean_absolute_error(y_test_inv, y_pred_log_inv)
rmse_log = np.sqrt(mean_squared_error(y_test_inv, y_pred_log_inv))
mape_log = mean_absolute_percentage_error(y_test_inv, y_pred_log_inv)
r2_log = r2_score(y_test_inv, y_pred_log_inv)

print("\nResultados con logaritmo:")
print(f"MAE de testeo fue: {mae_log}")
print(f"RMSE de testeo fue: {rmse_log}")
print(f"MAPE de testeo fue: {mape_log}")
print(f"R2 de testeo fue: {r2_log}")

/var/folders/c_/rq9m5qb17jsg0xbdp8j28n2c0000gn/T/ipykernel_36566/1589289974.py:5: RuntimeWarning: overflow encountered in exp
  y_pred_log_inv = np.exp(y_pred_log)


ValueError: Input contains infinity or a value too large for dtype('float64').

In [11]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

results_df = pd.DataFrame({
    'y_original': y_test,
    'y_pred_original': y_pred_original,
    'y_test_log': y_test_log,
    'y_pred_log': y_pred_log,
    'y_test_inv': y_test_inv,
    'y_pred_log_inv': y_pred_log_inv,
})

In [12]:
results_df

,y_original,y_pred_original,y_test_log,y_pred_log,y_test_inv,y_pred_log_inv
1971,198000,-9775.223,12.196,-9775.223,198000.000,0.000
4664,500000,665169.017,13.122,665169.017,500000.000,inf
5448,425000,595389.100,12.960,595389.100,425000.000,inf
3333,150000,55167.691,11.918,55167.691,150000.000,inf
2316,525000,438429.445,13.171,438429.445,525000.000,inf
...,...,...,...,...,...,...
462,600000,462910.673,13.305,462910.673,600000.000,inf
1956,400000,677720.674,12.899,677720.674,400000.000,inf
3782,500000,611733.052,13.122,611733.052,500000.000,inf
799,400000,406836.919,12.899,406836.919,400000.000,inf


In [34]:
y

0       450000
1       370000
2       158000
3       225000
4       130000
         ...  
8123    320000
8124    135000
8125    382000
8126    290000
8127    290000
Name: selling_price, Length: 8128, dtype: int64

In [35]:
y_log

0       13.017003
1       12.821258
2       11.970350
3       12.323856
4       11.775290
          ...    
8123    12.676076
8124    11.813030
8125    12.853176
8126    12.577636
8127    12.577636
Name: selling_price_log, Length: 8128, dtype: float64

In [66]:
preprocess_pipeline = Pipeline(steps=[
    ('name_spliter', NameSplitter()),
    ('mileage_converter', MileageConverter()),
    ('engine_converter', EngineConverter()),
    ('max_power_converter', MaxPowerConverter()),
    ('torque_standardizer', TorqueStandardizer()),
    ('map_owner', OwnerMapper()),
    ('model_dropper', ModelDropper()),
])

full_pipeline = Pipeline(steps=[
    ('multiple_iterative_imputer', MultipleIterativeImputer(columns_to_drop=columns_to_drop_for_imputation)),
    ('round_seats', SeatRounder()),
])

final_pipeline = Pipeline(steps=[
    ('preprocess_pipeline', preprocess_pipeline),
    ('full_pipeline', full_pipeline),
])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

regresion_lineal = LinearRegression()

sc_X = StandardScaler()
X_train_scaled = sc_X.fit_transform(X_train)
X_test_scaled = sc_X.transform(X_test)

regresion_lineal.fit(X_train_scaled, y_train)

y_pred_lineal = regresion_lineal.predict(X_test_scaled)

print(f"El error de testeo fue: {mean_absolute_error(y_test, y_pred_lineal)}")

- Ridge
- Arbol regresión
- SVR
- Boost (hay 2)
- Random Forest